In [4]:
import numpy as np
import math

There is a nice elementary proof of this series using the [Fibonacci generating function][1]. Consider the sum
$$
\begin{align*}
(1 - x - x^2)\sum_{n=1}^{\infty} F_{n}x^{n} &= \sum_{n=1}^{\infty} F_{n}x^{n} - \sum_{n=1}^{\infty} F_{n}x^{n+1} - \sum_{n=1}^{\infty} F_{n}x^{n+2} \\
&= \sum_{n=1}^{\infty} F_{n}x^{n} - \sum_{n=2}^{\infty} F_{n-1}x^{n} - \sum_{n=3}^{\infty} F_{n-2}x^{n} \\
&= \left[F_1 x + F_2 x^2 + \sum_{n=3}^{\infty} F_{n}x^{n} \right] - \left[F_{1}x^2 + \sum_{n=3}^{\infty} F_{n-1}x^{n}\right] - \sum_{n=3}^{\infty} F_{n-2}x^{n} \\
&= F_1 x + (F_2 - F_1) x^2 + \sum_{n=3}^{\infty} (F_{n} - F_{n-1} - F_{n-2})x^{n} \\
&= x 
\end{align*}
$$

The last line uses the definition of the Fibonacci sequece: $F_1 = F_2 = 1$ and that $F_{n} = F_{n-1} + F_{n-2}$ to show the second term and the summation are both $0$. So we get
$$
(1 - x - x^2)\sum_{n=1}^{\infty} F_{n}x^{n} = x \implies \sum_{n=1}^{\infty} F_{n}x^{n} = \frac{x}{1 - x - x^2}.
$$

If this must be an integer $k$, we get
$$
x = k(1 - x - x^2) \implies kx^2 + (k+1)x - k = 0 \implies x = \frac{-k-1 \pm \sqrt{(k+1)^2 + 4k^2}}{2}
$$
which can only have a rational solution if the discriminant is a perfect square. So we need to check each $k$ such that $(k+1)^2 + (2k)^2$ is a perfect square. 

A couple ways to describe the problem with some skills we did in previous problems:
1. This is basically generating a Pythagorean triple $(a,b,c)$ with the requirement that $a-1 = b/2$.
2. This is solving the Diophantine equation $x^2 + 4y^2 = z^2$ with the requirement $x = y+1$.

However, these are too slow. It turns out this is a [known series][2] and that any rational $x^*$ has an associated $n \geq 0$ such that $A_F(x^*) = F_{2n} \cdot F_{2n+1}$ is an integer, with $F_0 = 0$ and $F_1 = 1$. The proof is quite complicated from what I saw.


[1]: https://en.wikipedia.org/wiki/Fibonacci_sequence#Generating_function
[2]: https://oeis.org/A081018

In [14]:
def valid_ints(number_needed):
    fib = [0, 1]
    for i in range(2*number_needed + 1):
        fib.append(fib[-1] + fib[-2])

    return [fib[2*i]*fib[2*i + 1] for i in range(number_needed+1)]

In [16]:
valid_ints(number_needed=15)

[0,
 2,
 15,
 104,
 714,
 4895,
 33552,
 229970,
 1576239,
 10803704,
 74049690,
 507544127,
 3478759200,
 23843770274,
 163427632719,
 1120149658760]

In [15]:
valid_ints(number_needed=15)[15]

1120149658760

## Proof:

Let $(k+1)^2 + (2k)^2 = n^2$. Multiplying out the left hand side and multiplying both sides by $5$, we get
$$
5k^2 + 2k + 1 = n^2 \implies 25k^2 + 10k + 5 = 5n^2 \implies (25k^2 + 10k + 1) + 4 = 5n^2 \implies (5k + 1)^2 + 4 = 5n^2
$$

Substituting $m = 5k+1$, we get a [Pell equation][4]
$$
m^2 - 5n^2 = -4
$$

We can see quickly that the obvious integer solution is when $k = 2$, which implies $m = 11$ and $n = 5$. Let's first solve the related unit-form Pell equation
$$
x^2 - 5y^2 = 1
$$

It is known that the solution to Pell equations of the form $x^2 - Dy^2 = \pm 1$ can be solved using the [continued fraction][3] representation of $\sqrt{D}$. The continued fraction of $\sqrt{5}$ can be found using the following algorithm:

> Initialize $a_0 = \lfloor \sqrt{D} \rfloor$ and $f_0 = \sqrt{D} - a_0$. Then the next coefficients can be caluclated as
> $$
> a_n = \left\lfloor \frac{1}{f_{n-1}} \right\rfloor, \quad f_n = f_{n-1} - a_n
> $$

The coefficients are periodic for an irrational number, so suppose $r \in \mathbb{Z}$ such that $a_{r+1} = 2*a_0$. In this case we calculate the continued fraction representation of $\sqrt{5}$ is $[2; \bar{4}]$, so $r = 0$. Then we can get the convergent numerators $p_n$ and denominatos $q_n$ (and the related partial numerators and denominators $P_n$ and $Q_n$) using the following algorithm.
> Initialize $p_0 = a_0$ and $p_1 = a_0 a_1 + 1$. Initialize $q_0 = 1$ and $q_1 = a_1$. then the next convergents can be calculated as
> $$
> p_n = a_n p_{n-1} + p_{n-2}, \quad q_n = a_n q_{n-1} + q_{n-2}
> $$
> Initialize $P_0 = 0$ and $P_1 = a_0$. Initialize $Q_0 = 1$ and $Q_1 = D - a_0^2$. then the next partial numerators and denominators can be calculated as
> $$
> P_n = a_{n-1} Q_{n-1} - P_{n-1}, \quad Q_n = \frac{D - P_n^2}{Q_{n-1}}
> $$

Once we calculate the convergents, we can use the identity (can be proved using induction) $p_n^2 - Dq_n^2 = (-1)^{n+1} Q_{n+1}$ to get the smallest solutions
$$
(x,y) = \begin{cases} (p_r, q_r) & \text{if $r$ is odd} \\ (p_{2r+1}, q_{2r+1}) & \text{if $r$ is even} \end{cases}
$$

In this case, $r = 0$, so our smallest integer solution is $x_1 = p_{2r+1} = 9$ and $y_1 = q_{2r+1} = 4$. To expand to find a famly of solutions we can take the equation to the $n^{th}$ power. For example, in our case, suppose a new solution $(x_2,y_2)$ such that 
$$
\begin{align*}
x_2^2 - 5y_2^2 = (x_0^2 - 5y_0^2)^2 \implies &(x_2 - y_2\sqrt{5})(x_2 + y_2\sqrt{5}) = (x_1 - y_1 \sqrt{5})^2 (x_1 + y_1 \sqrt{5})^2 \\
    \implies &x_2 + y_2\sqrt{5} = (x_1 + y_1 \sqrt{5})^2 \quad \text{and} \quad x_2 - y_2 \sqrt{5} = (x_1 - y_1 \sqrt{5})^2 \\
    \implies &x_2 + y_2 \sqrt{5} = x_1^2 + 2\sqrt{5} x_1 y_1 + y_1^2 \\
    \implies &x_2 = x_1^2 + y_1^2 \quad \text{and} \quad y_2 = 2x_1 y_1
\end{align*}
$$

If we apply the same logic with induction, we will get the famous [recurrence relations][5] to generate additional solutions for the Pell equation:
$$
x_n = x_1 x_{n-1} + Dy_1 y_{n-1} \quad \text{and} \quad y_n = x_1 y_{n-1} + x_{n-1} y_1
$$

Finally, we can apply the solutions to the unit-form equation to our original equation as
$$
(m^2 - 5n^2)(x_n^2 - 5y_n^2) = -4 \cdot 1 = -4
$$

Iterating through these, we can generate all solutions that way too with the recurrence relation:
$$
m_k + n_k \sqrt{5} = (m_1 + n_1 \sqrt{5}) (x_k + y_k \sqrt{5})
$$

[3]: https://en.wikipedia.org/wiki/Continued_fraction#Calculating_continued_fraction_representations
[4]: https://mathworld.wolfram.com/PellEquation.html
[5]: https://en.wikipedia.org/wiki/Pell%27s_equation#Additional_solutions_from_the_fundamental_solution

$$
\begin{align*}
(m_1 + n_1 \sqrt{5}) (x_k + y_k \sqrt{5}) &= m_1 x_k + n_1 x_k \sqrt{5} + m_1 y_k \sqrt{5} + 5 n_1 y_k \\
    &= (m_1 x_k + 5 n_1 y_k) + (n_1 x_k + m_1 y_k) \sqrt{5}
\end{align*}
$$

In [30]:
# find fundamental solutions to m^2 - 5n^2 = -4
to_check = [(0,0)]
seen = set()
while to_check:
    m, n = to_check.pop(0)
    if abs(m + n) > 50:
        break

    if (m,n) in seen: continue
    seen.add((m,n))

    if m*m - 5*n*n == -4:
        print(m,n)
    
    to_check += [(m-1, n), (m, n-1), (m, n+1), (m+1, n)]

-1 -1
-1 1
1 -1
1 1
-4 -2
-4 2
4 -2
4 2
-11 -5
-11 5
11 -5
11 5
-29 -13
-29 13
29 -13
29 13


In [27]:
# use recurrence relation on fundamental solutions to generate 3 solution sets
x0,y0 = 1,0
x,y = x0, y0
for i in range(15):
    if (1*x + 5*1*y - 1)/5 == (1*x + 5*1*y - 1)//5: print(1*x + 5*1*y, 1*x + 1*y, (1*x + 5*1*y - 1)/5)
    if (4*x + 5*2*y - 1)/5 == (4*x + 5*2*y - 1)//5: print(4*x + 5*2*y, 2*x + 4*y, (4*x + 5*2*y - 1)/5)
    if (11*x + 5*5*y - 1)/5 == (11*x + 5*5*y - 1)//5: print(11*x + 5*5*y, 5*x + 11*y, (11*x + 5*5*y - 1)/5)
    
    if (x,y) == (1,0):
        x0, y0 = 9,4
    x,y = x*x0 + 5*y*y0, y*x0 + x*y0
    

1 1 0.0
11 5 2.0
76 34 15.0
521 233 104.0
3571 1597 714.0
24476 10946 4895.0
167761 75025 33552.0
1149851 514229 229970.0
7881196 3524578 1576239.0
54018521 24157817 10803704.0
370248451 165580141 74049690.0
2537720636 1134903170 507544127.0
17393796001 7778742049 3478759200.0
119218851371 53316291173 23843770274.0
817138163596 365435296162 163427632719.0
5600748293801 2504730781961 1120149658760.0
38388099893011 17167680177565 7677619978602.0
263115950957276 117669030460994 52623190191455.0
1803423556807921 806515533049393 360684711361584.0
12360848946698171 5527939700884757 2472169789339634.0
